In [4]:
library(ggplot2)
library(ggrepel)

In [6]:
library(here)
readRenviron(here("config.env"))
resultsdir <- Sys.getenv("resultsdir")
resultsdir

[1] "/data/user24/rhds/results"

In [7]:
combined.filename <- file.path(resultsdir, "combined-clin-pred-proteins.txt")
data <- read.table(combined.filename,
  header = T, sep = "\t", stringsAsFactors = F
)

protein.names <-
  subset(
    meffonym::meffonym.models(full = T),
    grepl("^episcores", filename)
  )$name
protein.names <- make.names(protein.names)

table(protein.names %in% colnames(data))

## Run glms for association between proteins levels and tissue type (tumor vs. normal)

## define glm formulae with pred.proteins as predictors of 'tumor.or.normal'
## tissue i.e. tumor.or.normal ~ pred.protein
formulae <- sapply(protein.names, function(i) {
  reformulate(i, response = "tumor")
}, simplify = F)

# run glms
fit <- sapply(formulae, function(i) {
  glm(i, data = data, family = binomial())
}, simplify = F)

fit.summary <- sapply(fit, function(i) {
  out <- summary(i)$coefficients
  out[, "Estimate"] <- out[, "Estimate"]
  out
}, simplify = F)

fit.coefs <- sapply(fit.summary, function(i) {
  i[2, c("Estimate", "Pr(>|z|)")]
}, simplify = F)
fit.coefs <- {
  x <- do.call(rbind, fit.coefs)
  data.frame(
    pred.protein = rownames(x),
    coef = x[, "Estimate"],
    p.value = x[, "Pr(>|z|)"]
  )
}

bonferroni <- -log10(0.05 / length(fit))


loading models ...
Wed Feb  4 18:41:00 2026 Adding model:  wid-smk-distal-epithelial-hyperM 
Wed Feb  4 18:41:00 2026 Adding model:  wid-smk-immune-hypoM 
Wed Feb  4 18:41:00 2026 Adding model:  wid-smk-epithelial-hypoM 
Wed Feb  4 18:41:00 2026 Adding model:  wid-smk-proximal-epithelial-hyperM 
Wed Feb  4 18:41:00 2026 Adding model:  wahl-bmi-european 
Wed Feb  4 18:41:00 2026 Adding model:  wahl-bmi-indian 
Wed Feb  4 18:41:00 2026 Adding model:  wahl-bmi 
Wed Feb  4 18:41:00 2026 Adding model:  gs-cognitive 
Wed Feb  4 18:41:00 2026 Adding model:  patella-lead 
Wed Feb  4 18:41:00 2026 Adding model:  tibia-lead 
Wed Feb  4 18:41:00 2026 Adding model:  cmv 
Wed Feb  4 18:41:00 2026 Adding model:  epismoker-current 
Wed Feb  4 18:41:00 2026 Adding model:  epismoker-former 
Wed Feb  4 18:41:00 2026 Adding model:  epismoker-never 
Wed Feb  4 18:41:00 2026 Adding model:  elliott-smoking 
Wed Feb  4 18:41:00 2026 Adding model:  zhang-smoking 
Wed Feb  4 18:41:00 2026 Adding model:  do.bmi


TRUE 
 109 

In [ ]:
fit.coefs |>
  ggplot(aes(x = pred.protein, y = -log10(p.value))) +
  geom_point() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1)) +
  geom_text_repel(
    data = fit.coefs[which((-log10(fit.coefs$p.value) > bonferroni)), ],
    aes(x = pred.protein, y = -log10(p.value), label = pred.protein)
  ) +
  geom_hline(
    yintercept = bonferroni,
    linetype = "dashed"
  )

fit.coefs |>
  ggplot(aes(x = coef, y = -log10(p.value))) +
  geom_point() +
  geom_text_repel(
    data = fit.coefs[which((-log10(fit.coefs$p.value) > bonferroni)), ],
    aes(x = coef, y = -log10(p.value), label = pred.protein)
  ) +
  geom_hline(
    yintercept = bonferroni,
    linetype = "dashed"
  )
